# BlueSprig-CV 蓝枝简历 — 简单demo演示
## 1. 概述
    本demo演示了本组使用蓝心大模型，通过构建prompt和message，实现了部分最终产品功能的python代码演示

主要功能
1. 解析word，pdf或者图片格式的简历，给出构建后的规范化简历json格式
2. 根据规范化简历json，生成当前简历各个模块的评分，优化后的版本以及优化建议，提供多轮对话，可以按照用户的想法继续优化
3. 根据规范化简历json，基于场景构建一场模拟面试，用语音播放题目，允许用户使用语音或文字回答问题，并在最终给出面试评分
    1. 面试场景可以分为：技术面试、产品面试、HR面试、技术+产品面试、技术+HR面试等等，后续可能根据不同行业标准进行调整，可以通过RAG减弱大模型幻觉现象，保证面试题更贴近真实面试，（本功能点因为时间问题当前demo并未实现）


## 2. 主要功能演示
### 1. 解析word，pdf或者图片格式的简历，给出构建后的规范化简历json格式

In [1]:
import json

# 工具方法
from util.file_loader import load_file
from util.llm import format_resume, better_resume_by_module_chat


# 定义一个函数，用于读取txt文件
def read_txt(file_path):
    # 打开文件，以只读模式，编码格式为utf-8
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取文件内容，并去除首尾空格
        return file.read().strip()

In [2]:
# 读取简历文件
resume_path = 'assets/pdf/1.pdf'
content = load_file(resume_path)
content

'张文博\n意向岗位: 后端开发。\n            \n            \n            \n\n电话：15602032933\n邮箱：Hotcocoa.Canary@outlook.com。\n            \n            \n            \n\n性别：男\n年龄：21。\n            \n            \n            \n\n户籍：内蒙古自治区\n政治面貌：共青团员。\n            \n            \n            \n\n民族：汉。\n            \n            \n            \n\n求职意向。\n            \n            \n            \n\n意向岗位: 后端开发\n意向城市: 北京。\n            \n            \n            \n\n期望薪资: 8k-20k\n求职类型: 校招。\n            \n            \n            \n\n期望行业: 计算机软件\n当前状态: 月内到岗。\n            \n            \n            \n\n教育经历。\n            \n            \n            \n\n内蒙古农业大学。\n            \n            \n            \n\n2022.09 - 2025.07。\n            \n            \n            \n\n软件工程\n本科\n|。\n            \n            \n            \n\n荣誉/奖项：。\n            \n            \n            \n\n• 自治区励志奖学金（2023-10）。\n            \n            \n            \n\n• 多次获得校级一等优秀学生（2024-5，2024-11）。\n            \n            \n    

In [3]:
# 获取标准解析简历json格式
resume_format = read_txt('assets/txt/resume_format.txt')
response_content = format_resume(resume_format, content)
response_content

'{"job_intent_info":[{"job_intention":"后端开发","expected_salary":"8k-20k","preferred_city":"北京","job_industry":"计算机软件","job_type":"校招"}],"basic_info":{"name":"张文博","age":"21","sex":"男","native_place":"内蒙古自治区","education":"本科","if_party_membership":"共青团员","marital_status":"","phone_number":"15602032933","email":"Hotcocoa.Canary@outlook.com","if_fresh_graduate":"是","years_of_work_experience":"","expected_salary":""},"work_experience":[{"job_position":"大模型开发实习生","work_period":"2024.02 - 2024.10","company_name":"北京以塔科技有限公司","work_content":"• 智能简历解析系统开发。\\n• Context：应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。\\n• Task：构建基于大模型的简历结构化处理与诊断系统。\\n• Action：采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，\\n简历优化）。\\n• Result：完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，\\n对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。\\n• RAG增强问答系统开发。\\n•  Context：构建基于pdf检索增强生成的问答系统。\\n• Task：实现文档智能解析与语义检索核心模块。\\n• Action：基于LangChain搭建处理流水线，设计混合检索策略：关键词匹配 + 语义检索（ChromaDB向量库），实\\n现上下文感知问答：通过Prompt工程优化大模型响应准确。\\n• Res

### 3. 根据规范化简历json，生成当前简历各个模块的评分，优化后的版本以及优化建议，提供多轮对话，可以按照用户的想法继续优化

#### 3.1. 简历优化版本

In [4]:
# 简历json数据
resume_json = json.loads(response_content)
resume_json

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'job_position': '大模型开发实习生',
   'work_period': '2024.02 - 2024.10',
   'company_name': '北京以塔科技有限公司',
   'work_content': '• 智能简历解析系统开发。\n• Context：应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。\n• Task：构建基于大模型的简历结构化处理与诊断系统。\n• Action：采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，\n简历优化）。\n• Result：完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，\n对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。\n• RAG增强问答系统开发。\n•  Context：构建基于pdf检索增强生成的问答系统。\n• Task：实现文档智能解析与语义检索核心模块。\n• Action：基于LangChain搭建处理流水

In [5]:
from util.llm import better_resume_by_module


work_experience_content = resume_json["work_experience"]
education_experience_content = resume_json["education_experience"]
project_experience_content = resume_json["project_experience"]

work_experience_format = read_txt("assets/txt/greater_resume_module/work_experience.txt")
education_experience_format = read_txt("assets/txt/greater_resume_module/education_experience.txt")
project_experience_format = read_txt("assets/txt/greater_resume_module/project_experience.txt")

work = better_resume_by_module(work_experience_format, work_experience_content, "工作经历")
education = better_resume_by_module(education_experience_format, education_experience_content, "教育经历")
project = better_resume_by_module(project_experience_format, project_experience_content, "项目经历")

project = json.loads(project)
work = json.loads(work)
education = json.loads(education)

resume_json["project_experience"] = project
resume_json["work_experience"] = work
resume_json["education_experience"] = education

resume_json

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'company': '北京以塔科技有限公司',
   'position': '大模型开发实习生',
   'location': '',
   'duration': '2024.02 - 2024.10',
   'responsibilities': [{'item': '智能简历解析系统开发',
     'description': '应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。构建基于大模型的简历结构化处理与诊断系统。采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，简历优化）。完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。'},
    {'item': 'RAG增强问答系统开发',
     'description': '构建基于pdf检索增强生成的问答系统。实现文档智能解析与语义检索核心模块。基于LangChain搭建处理流水线

#### 3.2. 简历优化对话

In [6]:
# 获取原始简历
resume_json_old = json.loads(response_content)
resume_json_old

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'job_position': '大模型开发实习生',
   'work_period': '2024.02 - 2024.10',
   'company_name': '北京以塔科技有限公司',
   'work_content': '• 智能简历解析系统开发。\n• Context：应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。\n• Task：构建基于大模型的简历结构化处理与诊断系统。\n• Action：采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，\n简历优化）。\n• Result：完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，\n对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。\n• RAG增强问答系统开发。\n•  Context：构建基于pdf检索增强生成的问答系统。\n• Task：实现文档智能解析与语义检索核心模块。\n• Action：基于LangChain搭建处理流水

In [7]:
# 获取优化后的简历
resume_json_new = resume_json
resume_json_new

{'job_intent_info': [{'job_intention': '后端开发',
   'expected_salary': '8k-20k',
   'preferred_city': '北京',
   'job_industry': '计算机软件',
   'job_type': '校招'}],
 'basic_info': {'name': '张文博',
  'age': '21',
  'sex': '男',
  'native_place': '内蒙古自治区',
  'education': '本科',
  'if_party_membership': '共青团员',
  'marital_status': '',
  'phone_number': '15602032933',
  'email': 'Hotcocoa.Canary@outlook.com',
  'if_fresh_graduate': '是',
  'years_of_work_experience': '',
  'expected_salary': ''},
 'work_experience': [{'company': '北京以塔科技有限公司',
   'position': '大模型开发实习生',
   'location': '',
   'duration': '2024.02 - 2024.10',
   'responsibilities': [{'item': '智能简历解析系统开发',
     'description': '应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。构建基于大模型的简历结构化处理与诊断系统。采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，简历优化）。完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。'},
    {'item': 'RAG增强问答系统开发',
     'description': '构建基于pdf检索增强生成的问答系统。实现文档智能解析与语义检索核心模块。基于LangChain搭建处理流水线

In [9]:
# 简历信息输出以及优化对话窗口
BETTER_RESUME_MOD_LIST = ["work_experience", "education_experience", "project_experience"]

# 遍历resume_json_old中的每个键值对
for key, value in resume_json_old.items():
    # 如果键不在BETTER_RESUME_MOD_LIST中，直接输出
    if key not in BETTER_RESUME_MOD_LIST:
        print(f"{key}模块信息如下：")
        print(f"{value}")
        print("#######################################################")
    # 如果键在BETTER_RESUME_MOD_LIST中，输出old原有的值
    else:
        print(f"{key}模块信息如下：")
        print(f"{value}")
        print(f"{key}模块初步优化后的结果如下，如果有问题或者建议可以直接指正，没有请输入“没有问题”，我们将进行下一个模块的优化")
        print(f"{key} (new): {resume_json_new[key]}")
        message=[
            {
                "role": "user",
                "content": f"{key}模块信息如下："
            },
            {
                "role": "assistant",
                "content": f"{key}模块初步优化后的结果如下，如果有问题或者建议可以直接指正，我将会按照你的要求重新优化"+str(resume_json_new[key])
            }
        ]
        while input()!= "没有问题":
            message.append({
                "role": "user",
                "content": input()
            })
            format_content = read_txt("assets/txt/greater_resume_module/"+key+".txt")
            better_part_content = better_resume_by_module_chat(format_content, message, value)
            message.append({
                "role": "assistant",
                "content": f"按照你的要求，优化后的结果如下，如果有问题或者建议可以直接指正，我将会按照你的要求重新优化"+better_part_content
            })
            print(f"按照你的要求，优化后的结果如下，如果有问题或者建议可以直接指正，我将会按照你的要求重新优化"+better_part_content)
        print("#######################################################")

job_intent_info模块信息如下：
[{'job_intention': '后端开发', 'expected_salary': '8k-20k', 'preferred_city': '北京', 'job_industry': '计算机软件', 'job_type': '校招'}]
#######################################################
basic_info模块信息如下：
{'name': '张文博', 'age': '21', 'sex': '男', 'native_place': '内蒙古自治区', 'education': '本科', 'if_party_membership': '共青团员', 'marital_status': '', 'phone_number': '15602032933', 'email': 'Hotcocoa.Canary@outlook.com', 'if_fresh_graduate': '是', 'years_of_work_experience': '', 'expected_salary': ''}
#######################################################
work_experience模块信息如下：
[{'job_position': '大模型开发实习生', 'work_period': '2024.02 - 2024.10', 'company_name': '北京以塔科技有限公司', 'work_content': '• 智能简历解析系统开发。\n• Context：应对企业多格式简历智能处理需求，设计可扩展的AI解析引擎。\n• Task：构建基于大模型的简历结构化处理与诊断系统。\n• Action：采用大模型Prompt工程针对项目接口进行开发（包含：简历诊断，pdf简历信息提取，图片简历信息提取，\n简历优化）。\n• Result：完成大模型返回值提取规范化json功能，实现业务核心逻辑简历诊断及评分，实现业务核心逻辑简历优化，\n对大模型Prompt的调试更加得心应手，对与要求大模型返回值的规范化处理积累一定经验，熟悉git开发流程。\n• RAG增强问答系统开发。\n•  Contex

KeyboardInterrupt: Interrupted by user